In [77]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
import os

try:
    driver = webdriver.Chrome()
except:
    print("AF: No Chrome webdriver installed")
    driver = webdriver.Chrome(ChromeDriverManager().install())

try:
    url = "https://www.forexfactory.com/calendar?month=jan.2008"
    driver.get(url)
    while True:
        before_scroll = driver.execute_script("return window.pageYOffset;")
        driver.execute_script("window.scrollTo(0, window.pageYOffset + 500);")
        time.sleep(2)
        after_scroll = driver.execute_script("return window.pageYOffset;")
        if before_scroll == after_scroll:
            break

    # Get the table
    table = driver.find_element(By.CLASS_NAME, "calendar__table")
    data = []
    last_date = None
    last_time = None
    currency_codes = ['CAD', 'JPY', 'USD', 'GBP', 'EUR', 'NZD', 'AUD', 'CHF', 'CNY']

    for row in table.find_elements(By.TAG_NAME, "tr"):
        row_data = []
        for td in row.find_elements(By.TAG_NAME, "td"):
            if td.text:
                text = td.text.replace('\n', ', ')
                row_data.append(text)
                if ',' in text:  
                    last_date = text
                elif re.match(r'\d+:\d+[ap]m', text):
                    last_time = text
            else:
                if td.get_attribute("class") == "calendar__cell calendar__impact":
                    span_element = td.find_element(By.TAG_NAME, 'span')
                    row_data.append(span_element.get_attribute("title"))
        row_data = [x for x in row_data if x is not None] 
        if row_data and len(row_data) > 1:  
            if row_data[0] == 'Tentative':  
                row_data.insert(0, last_time)  
                row_data.insert(0, last_date)  
            elif row_data[0].startswith('Day'):  
                row_data.insert(0, last_date)  
            elif row_data[0] in currency_codes:  
                row_data.insert(0, last_time)  
                row_data.insert(0, last_date)  
            elif row_data[0] == 'All Day' or re.match(r'\d+:\d+[ap]m', row_data[0]):  
                row_data.insert(0, last_date)  
            data.append(row_data)
            print(row_data)

    second_list = ['CAD', 'JPY', 'USD', 'GBP', 'EUR', 'NZD', 'AUD', 'CHF', 'CNY']
    result_list = [[None] + row if len(row) > 1 and row[1] in second_list else row for row in data]
    result_list = [[None] + [None] + row if len(row) > 0 and row[0] in second_list else row for row in result_list]

    # Ensure each row has 8 elements
    for i in range(len(result_list)):
        while len(result_list[i]) < 8:
            result_list[i].append('')

    column_names = ['Date', 'Time', 'Currency', 'Impact', 'Event', 'Actual', 'Forecast', 'Previous']
    df = pd.DataFrame(result_list)
    df.columns = column_names

    month_year = url.split('=')[-1]
    year = month_year.split('.')[-1]

    directory = f'data/{year}'
    if not os.path.exists(directory):
        os.makedirs(directory)

    df.to_csv(f'{directory}/{month_year}.csv', index=False)
    print("done")

except Exception as e:
    print(e)

finally:
    driver.quit()

['Sat, Dec 1', '1:40am', 'USD', 'Medium Impact Expected', 'FOMC Member Kroszner Speaks']
['Sat, Dec 1', '8:00am', 'CNY', 'High Impact Expected', 'Manufacturing PMI', '55.4', '53.2']
['Mon, Dec 3', '5:30am', 'AUD', 'Low Impact Expected', 'AIG Manufacturing Index', '53.8', '53.3']
['Mon, Dec 3', '6:30am', 'AUD', 'Low Impact Expected', 'MI Inflation Gauge m/m', '0.3%', '0.3%']
['Mon, Dec 3', '6:50am', 'JPY', 'Medium Impact Expected', 'Capital Spending q/y', '-1.2%', '-2.5%', '-4.9%']
['Mon, Dec 3', '7:30am', 'AUD', 'High Impact Expected', 'Goods Trade Balance', '-2.98B', '-1.88B', '-1.92B']
['Mon, Dec 3', '7:30am', 'AUD', 'Low Impact Expected', 'Company Operating Profits q/q', '-2.1%', '1.9%', '1.7%']
['Mon, Dec 3', '8:00am', 'CNY', 'Low Impact Expected', 'Non-Manufacturing PMI', '54.1', '65.0']
['Mon, Dec 3', '8:30am', 'JPY', 'Low Impact Expected', 'Average Cash Earnings y/y', '0.0%', '0.0%', '-0.6%']
['Mon, Dec 3', '9:30am', 'CNY', 'Medium Impact Expected', 'Caixin Manufacturing PMI', '

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re
import os

try:
    driver = webdriver.Chrome()
except:
    print("AF: No Chrome webdriver installed")
    driver = webdriver.Chrome(ChromeDriverManager().install())

months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
years = range(2008, 2025)  # 2025 is exclusive, so it will stop at 2024

try:
    for year in years:
        for month in months:
            url = f"https://www.forexfactory.com/calendar?month={month}.{year}"
            driver.get(url)
            while True:
                before_scroll = driver.execute_script("return window.pageYOffset;")
                driver.execute_script("window.scrollTo(0, window.pageYOffset + 500);")
                time.sleep(2)
                after_scroll = driver.execute_script("return window.pageYOffset;")
                if before_scroll == after_scroll:
                    break

            wait = WebDriverWait(driver, 60)
            table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "calendar__table")))

            data = []
            last_date = None
            last_time = None
            currency_codes = ['CAD', 'JPY', 'USD', 'GBP', 'EUR', 'NZD', 'AUD', 'CHF', 'CNY']

            for row in table.find_elements(By.TAG_NAME, "tr"):
                row_data = []
                for td in row.find_elements(By.TAG_NAME, "td"):
                    if td.text:
                        text = td.text.replace('\n', ', ')
                        row_data.append(text)
                        if ',' in text:  
                            last_date = text
                        elif re.match(r'\d+:\d+[ap]m', text):
                            last_time = text
                    else:
                        if td.get_attribute("class") == "calendar__cell calendar__impact":
                            span_element = td.find_element(By.TAG_NAME, 'span')
                            row_data.append(span_element.get_attribute("title"))
                row_data = [x for x in row_data if x is not None] 
                if row_data and len(row_data) > 1:  
                    if row_data[0] == 'Tentative':  
                        row_data.insert(0, last_time)  
                        row_data.insert(0, last_date)  
                    elif row_data[0].startswith('Day'):  
                        row_data.insert(0, last_date)  
                    elif row_data[0] in currency_codes:  
                        row_data.insert(0, last_time)  
                        row_data.insert(0, last_date)  
                    elif row_data[0] == 'All Day' or re.match(r'\d+:\d+[ap]m', row_data[0]):  
                        row_data.insert(0, last_date)  
                    data.append(row_data)
                    print(row_data)

            second_list = ['CAD', 'JPY', 'USD', 'GBP', 'EUR', 'NZD', 'AUD', 'CHF', 'CNY']
            result_list = [[None] + row if len(row) > 1 and row[1] in second_list else row for row in data]
            result_list = [[None] + [None] + row if len(row) > 0 and row[0] in second_list else row for row in result_list]

            # Ensure each row has 8 elements
            for i in range(len(result_list)):
                while len(result_list[i]) < 8:
                    result_list[i].append('')

            column_names = ['Date', 'Time', 'Currency', 'Impact', 'Event', 'Actual', 'Forecast', 'Previous']
            df = pd.DataFrame(result_list)
            df.columns = column_names

            directory = f'data/{year}'
            if not os.path.exists(directory):
                os.makedirs(directory)

            df.to_csv(f'{directory}/{month}.{year}.csv', index=False)
            print(f"Data for {month} {year} saved.")

except Exception as e:
    print(e)

finally:
    driver.quit()